source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path : "abfss://working-labs@datalakestorageaccountname.dfs.core.windows.net/bronze/geo-location/

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

### source and destination folders

In [0]:
# source_data = 'https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json'
source_base_url = 'https://geocoding-api.open-meteo.com/v1/search?name='
source_relative_url = '&count=10&language=en&format=json'
sink_layer = 'bronze'
sink_folder = 'geo-data'
sink_account_name ='madhupavanadls'
destination_location = f'abfss://{sink_layer}@{sink_account_name}.dfs.core.windows.net/{sink_folder}'


In [0]:
# schema = StructType([
#     StructField("id", IntegerType(), True),
#     StructField("name", StringType(), True),
#     StructField("latitude", DoubleType(), True),
#     StructField("longitude", DoubleType(), True),
#     StructField("elevation", IntegerType(), True),
#     StructField("feature_code", StringType(), True),
#     StructField("country_code", StringType(), True),
#     StructField("admin1_id", IntegerType(), True),
#     StructField("admin2_id", IntegerType(), True),
#     StructField("admin3_id", IntegerType(), True),
#     StructField("timezone", StringType(), True),
#     StructField("population", IntegerType(), True),
#     StructField("country_id", IntegerType(), True),
#     StructField("country", StringType(), True),
#     StructField("admin1", StringType(), True),
#     StructField("admin2", StringType(), True),
#     StructField("admin3", StringType(), True)
# ])

In [0]:
full_schema = StructType([
    StructField("results", schema, True),
    StructField("generationtime_ms", DoubleType(), True)
])

In [0]:
import requests
import json
import pandas as pd

data = requests.get(source_data).json()
pd_df = pd.DataFrame(data)
spark_df = spark.createDataFrame(pd_df,full_schema).select(col('results.*'))


### get the market_names from the dim_market_table from gold layer

In [0]:
Sql_query = '''
with cte as (
select 
trim(replace(lower(market_name),'"',"")) as actual_str,
instr(trim(replace(lower(market_name),'"',"")),"(") as brck_ind
from uc_prod.gold.reporting_dim_market_gold
)
select 
case when brck_ind > 0 then left(actual_str,brck_ind-1) else actual_str end
as market_name from cte
'''

In [0]:
market_df = spark.sql(Sql_query).collect()
market_list = [row['market_name'] for row in market_df]
json_array_data = []
for item in market_list:
    url_is = source_base_url + item+ source_relative_url
    data = requests.get(url_is).json()
    if isinstance(data, dict):
        json_array_data.append(data)
pd_df = pd.DataFrame(json_array_data)
spark_df = spark.createDataFrame(pd_df)

### writing the into bronze

In [0]:
spark_df.filter(col('results').isNotNull()).write.format('json').mode('overwrite').save(destination_location)
# spark_df.display()

## checking